In [1]:
%load_ext autotime

In [2]:
import boto3
import datetime
import logging
import json
import pickle
import pytest
import sys
import time

from etltools import s3

from lambda_client import (
    ClaimsClient,
    BenefitsClient,
    CalculatorClient,
)

reload(logging)  # get around notebook problem

<module 'logging' from '/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/logging/__init__.pyc'>

time: 1.55 s


In [3]:
logging.basicConfig(
    level=logging.INFO, 
    format='[%(asctime)s] {%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
#     handlers=[
#         logging.FileHandler(filename='mylog.log', mode='w'),
#         logging.StreamHandler(sys.stdout),
#     ]
)

time: 2.27 ms


In [4]:
# Test whether logging works:
logger = logging.getLogger()
logger.info('TEST INFO')

[2018-01-18 17:27:37,369] {<ipython-input-4-efd2929c73d3>:3} INFO - TEST INFO


time: 2.05 ms


In [5]:
aws_info = {
    'profile_name': 'sandbox',
}

uids = s3.read_json('s3://picwell.sandbox.medicare/samples/philadelphia-2015-1k-sample')
pids = ['2820028008119', '2820088001036']

time: 790 ms


# Test ConfigInfo

In [6]:
from lambda_client.config_info import ConfigInfo

configs = ConfigInfo('lambda_client/lambda.cfg')

print configs.claims_bucket
print configs.claims_path
print
print configs.benefits_bucket
print configs.benefits_path
print
print configs.claims_table

picwell.sandbox.analytics
junghoon/lambda_calculator

picwell.sandbox.analytics
junghoon/lambda_calculator_benefits

ma_claims
time: 3.76 ms


In [7]:
all_states = configs.all_states

print '{} states'.format(len(all_states))
print all_states

51 states
['01', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56', '72']
time: 1.3 ms


# Test ClaimsClient

In [8]:
# Test S3:
client = ClaimsClient(aws_info, 
                      s3_bucket=configs.claims_bucket,
                      s3_path=configs.claims_path)

people = client.get(uids[:1])
print 'claims of {} people retrieved'.format(len(people))

claims of 1 people retrieved
time: 242 ms


In [9]:
person = people[0]
print person.keys()
{
    'uid': person['uid'],
    'medical_claims': person['medical_claims'][:5]
}

['medical_claims', 'uid']


{'medical_claims': [{'admitted': '2014-04-03',
   'benefit_category': 16,
   'cost': 148.0,
   'discharged': '2014-04-03',
   'length_of_stay': 1},
  {'admitted': '2014-05-02',
   'benefit_category': 16,
   'cost': 74.55,
   'discharged': '2014-05-02',
   'length_of_stay': 1},
  {'admitted': '2014-05-05',
   'benefit_category': 16,
   'cost': 104.39,
   'discharged': '2014-05-05',
   'length_of_stay': 1},
  {'admitted': '2014-05-13',
   'benefit_category': 16,
   'cost': 210.12,
   'discharged': '2014-05-13',
   'length_of_stay': 1},
  {'admitted': '2014-05-19',
   'benefit_category': 11,
   'cost': 442.2,
   'discharged': '2014-05-19',
   'length_of_stay': 1}],
 'uid': '1302895801'}

time: 5.3 ms


In [10]:
# Let's try something larger:
people = client.get(uids)
print 'claims of {} people retrieved'.format(len(people))

claims of 1000 people retrieved
time: 2min 28s


In [11]:
# Test DynamoDB:
client = ClaimsClient(aws_info,
                      table_name=configs.claims_table)

people = client.get(uids[:1])
print 'claims of {} people retrieved'.format(len(people))

claims of 1 people retrieved
time: 184 ms


In [12]:
person = people[0]
print person.keys()
{
    'uid': person['uid'],
    'medical_claims': person['medical_claims'][:5]
}

['medical_claims', 'uid']


{'medical_claims': [{'admitted': '2014-04-03',
   'benefit_category': 16,
   'cost': 148.0,
   'discharged': '2014-04-03',
   'length_of_stay': 1},
  {'admitted': '2014-05-02',
   'benefit_category': 16,
   'cost': 74.55,
   'discharged': '2014-05-02',
   'length_of_stay': 1},
  {'admitted': '2014-05-05',
   'benefit_category': 16,
   'cost': 104.39,
   'discharged': '2014-05-05',
   'length_of_stay': 1},
  {'admitted': '2014-05-13',
   'benefit_category': 16,
   'cost': 210.12,
   'discharged': '2014-05-13',
   'length_of_stay': 1},
  {'admitted': '2014-05-19',
   'benefit_category': 11,
   'cost': 442.2,
   'discharged': '2014-05-19',
   'length_of_stay': 1}],
 'uid': '1302895801'}

time: 4.72 ms


In [13]:
# Let's try something larger:
people = client.get(uids)
print 'claims of {} people retrieved'.format(len(people))

claims of 1000 people retrieved
time: 8.71 s


In [14]:
# Test configuration file and retrieving multiple people:
client = ClaimsClient(aws_info)

people = client.get(uids[:5])
print 'claims of {} people retrieved'.format(len(people))

claims of 5 people retrieved
time: 714 ms


In [15]:
# The object should not be pickled.
with pytest.raises(Exception, match='ClaimsClient object cannot be pickled.'):
    pickle.dumps(client)

time: 1.32 ms


# Test BenefitsClient

In [16]:
client = BenefitsClient(aws_info)

print client.all_states

['01', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56', '72']
time: 1.4 ms


In [17]:
plans = client._get_one_state('01')
print '{} plans read for state 01'.format(len(plans))

plans = client._get_one_state('04')
print '{} plans read for state 04'.format(len(plans))

47 plans read for state 01
75 plans read for state 04
time: 708 ms


In [18]:
plans = client.get_by_state(['01', '04'])
print '{} plans read'.format(len(plans))

122 plans read
time: 481 ms


In [19]:
plans = client.get_all()
print '{} plans read'.format(len(plans))

3558 plans read
time: 8.82 s


In [20]:
# Compare the timing against reading the entire file:
from lambda_client.shared_utils import _read_json

session = boto3.Session(**aws_info)
resource = session.resource('s3')

time: 60.9 ms


In [21]:
all_plans = _read_json('picwell.sandbox.medicare', 'ma_benefits/cms_2018_pbps_20171005.json', resource)

print '{} plans read'.format(len(plans))

3558 plans read
time: 2.99 s


In [22]:
# Ensure that the same plans are read:
sort_key = lambda plan: plan['picwell_id']
assert sorted(all_plans, key=sort_key) == sorted(plans, key=sort_key)

time: 141 ms


In [23]:
# The object should not be pickled.
with pytest.raises(Exception, match='BenefitsClient object cannot be pickled.'):
    pickle.dumps(client)

time: 1.41 ms


# Test Cost Breakdown

In [24]:
client = CalculatorClient(aws_info)

time: 599 µs


In [25]:
responses = client.get_breakdown(uids[:1], pids, verbose=True)

print '{} responses returned'.format(len(responses))
responses[0]

START RequestId: 36d8d96c-fc9f-11e7-9e99-9f968b194494 Version: $LATEST
[INFO]	2018-01-18T22:30:41.106Z	36d8d96c-fc9f-11e7-9e99-9f968b194494	Clock started at 2018-01-18 22:30:41.106150.
[INFO]	2018-01-18T22:30:41.131Z	36d8d96c-fc9f-11e7-9e99-9f968b194494	Thread initialization took 0.024793 seconds.
[INFO]	2018-01-18T22:30:41.212Z	36d8d96c-fc9f-11e7-9e99-9f968b194494	Found credentials in environment variables.
[INFO]	2018-01-18T22:30:41.672Z	36d8d96c-fc9f-11e7-9e99-9f968b194494	Starting new HTTPS connection (1): s3.amazonaws.com
[INFO]	2018-01-18T22:30:41.783Z	36d8d96c-fc9f-11e7-9e99-9f968b194494	Joining all threads took 0.651865 seconds.
[INFO]	2018-01-18T22:30:41.783Z	36d8d96c-fc9f-11e7-9e99-9f968b194494	Combining all results in order took 2.6e-05 seconds.
[INFO]	2018-01-18T22:30:41.783Z	36d8d96c-fc9f-11e7-9e99-9f968b194494	Claim retrieval for [u'1302895801'] took 0.677302 seconds.
[INFO]	2018-01-18T22:30:41.851Z	36d8d96c-fc9f-11e7-9e99-9f968b194494	Thread initialization took 0.067441 

{u'allowed': 26376.640000000003,
 u'covered_breakdown': {u'categories': {u'0': 0.0,
   u'11': 186.33999999999997,
   u'13': 13.23,
   u'15': 30.79,
   u'16': 0.0,
   u'19': 80.0,
   u'25': 1750.0,
   u'30': 719.1700000000001,
   u'31': 300.0,
   u'44': 0.0,
   u'49': 118.48800000000001,
   u'7': 479.558},
  u'composite': 3677.5759999999996,
  u'in_network': 3677.5759999999996,
  u'out_network': 0.0},
 u'deductible_breakdown': {u'categories': {u'0': 0.0,
   u'11': 0.0,
   u'13': 0.0,
   u'15': 0.0,
   u'16': 0.0,
   u'19': 0.0,
   u'25': 0.0,
   u'30': 0.0,
   u'31': 0.0,
   u'44': 0.0,
   u'49': 0.0,
   u'7': 0.0},
  u'composite': 0.0,
  u'in_network': 0.0,
  u'out_network': 0.0},
 u'oop': 3677.5759999999996,
 u'picwell_id': u'2820028008119',
 u'uid': u'1302895801',
 u'uncovered': 0.0,
 u'uncovered_breakdown': {u'categories': {u'0': 0.0,
   u'11': 0.0,
   u'13': 0.0,
   u'15': 0.0,
   u'16': 0.0,
   u'19': 0.0,
   u'25': 0.0,
   u'30': 0.0,
   u'31': 0.0,
   u'44': 0.0,
   u'49': 0.0,


time: 3.99 s


In [26]:
responses = client.get_breakdown(uids[:1], pids, use_s3_for_claims=False, verbose=True)

print '{} responses returned'.format(len(responses))

START RequestId: 392dc58a-fc9f-11e7-8487-bb60a367ee6d Version: $LATEST
[INFO]	2018-01-18T22:30:44.751Z	392dc58a-fc9f-11e7-8487-bb60a367ee6d	Clock started at 2018-01-18 22:30:44.751086.
[INFO]	2018-01-18T22:30:44.791Z	392dc58a-fc9f-11e7-8487-bb60a367ee6d	Thread initialization took 0.039802 seconds.
[INFO]	2018-01-18T22:30:44.793Z	392dc58a-fc9f-11e7-8487-bb60a367ee6d	Found credentials in environment variables.
[INFO]	2018-01-18T22:30:45.72Z	392dc58a-fc9f-11e7-8487-bb60a367ee6d	Starting new HTTPS connection (1): dynamodb.us-east-1.amazonaws.com
[INFO]	2018-01-18T22:30:45.474Z	392dc58a-fc9f-11e7-8487-bb60a367ee6d	Joining all threads took 0.68263 seconds.
[INFO]	2018-01-18T22:30:45.474Z	392dc58a-fc9f-11e7-8487-bb60a367ee6d	Combining all results in order took 2.3e-05 seconds.
[INFO]	2018-01-18T22:30:45.474Z	392dc58a-fc9f-11e7-8487-bb60a367ee6d	Claim retrieval for [u'1302895801'] took 0.723349 seconds.
[INFO]	2018-01-18T22:30:45.512Z	392dc58a-fc9f-11e7-8487-bb60a367ee6d	Thread initialization 

In [27]:
# Test recursive call:
responses = client.get_breakdown(uids[:10], pids, max_calculated_uids=10)

print '{} responses returned'.format(len(responses))

20 responses returned
time: 24.7 s


In [28]:
responses = client.get_breakdown(uids[:10], pids, max_lambda_calls=2)

print '{} responses returned'.format(len(responses))

20 responses returned
time: 10.7 s


In [29]:
responses = client.get_breakdown(uids[:10], pids)

print '{} responses returned'.format(len(responses))

20 responses returned
time: 6.45 s


In [30]:
# Check whether DynamoDB reduces latency:
responses = client.get_breakdown(uids[:10], pids, use_s3_for_claims=False, max_calculated_uids=10)

print '{} responses returned'.format(len(responses))

20 responses returned
time: 3.99 s


In [31]:
# Let's try something larger:
responses = client.get_breakdown(uids, pids)

print '{} responses returned'.format(len(responses))

2000 responses returned
time: 24.9 s


In [32]:
responses = client.get_breakdown(uids, pids, use_s3_for_claims=False)

print '{} responses returned'.format(len(responses))

2000 responses returned
time: 20.1 s


In [33]:
# Runs into memory issue if all 1000 people are calculated once:
responses = client.get_breakdown(uids, pids, use_s3_for_claims=False, max_calculated_uids=100, max_lambda_calls=10)

print '{} responses returned'.format(len(responses))

2000 responses returned
time: 15.3 s


In [45]:
from lambda_package.calc.calculator import calculate_oop

def run_locally(people, plans, oop_only):
    costs = []
    
    for person in people:
        claims = person['medical_claims']
    
        for plan in plans:

            cost = calculate_oop(claims, plan)
            if oop_only:
                cost = {
                    'oop': cost['oop']
                }

            cost.update({
                'uid': person['uid'],
                'picwell_id': str(plan['picwell_id']),
            })

            costs.append(cost)
    
    return costs

time: 6.28 ms


In [46]:
# Run calculcations locally for comparison:
# claims_client = ClaimsClient(aws_info, 
#                              s3_bucket=configs.claims_bucket,
#                              s3_path=configs.claims_path)
claims_client = ClaimsClient(aws_info, 
                             table_name=configs.claims_table)
people = claims_client.get(uids)

benefits_client = BenefitsClient(aws_info)
plans = benefits_client.get_by_pid(pids)

costs = run_locally(people, plans, False)

print '{} costs calculated'.format(len(costs))

2000 costs calculated
time: 11.2 s


In [35]:
# benefits_client = BenefitsClient()
benefits_client = BenefitsClient(aws_info)
plans_CA = benefits_client.get_by_state(['06'])
pids_CA = [plan['picwell_id'] for plan in plans_CA]

print '{} plans identified'.format(len(pids_CA))

268 plans identified
time: 353 ms


In [36]:
# Try a sample size more relevant to commercial:
responses = client.get_oop(uids[:300], pids_CA)

print '{} responses returned'.format(len(responses))

80400 responses returned
time: 19.4 s


In [37]:
responses = client.get_oop(uids[:300], pids_CA, use_s3_for_claims=False)

print '{} responses returned'.format(len(responses))

80400 responses returned
time: 17.5 s


In [42]:
# Increasing the amount of computation at the terminal nodes increases the time. This is probably
# because there are many plans.
responses = client.get_oop(uids[:300], pids_CA, use_s3_for_claims=False, max_calculated_uids=3)

print '{} responses returned'.format(len(responses))

80400 responses returned
time: 23.8 s


In [47]:
claims_client = ClaimsClient(aws_info, 
                             table_name=configs.claims_table)
people = claims_client.get(uids[:300])

benefits_client = BenefitsClient(aws_info)
plans = benefits_client.get_by_pid(pids_CA)

costs = run_locally(people, plans, True)

print '{} costs calculated'.format(len(costs))

80400 costs calculated
time: 1min 4s


# Test Batch Calculation

In [39]:
# uids = s3.read_json('s3n://picwell.sandbox.medicare/samples/philadelphia-2015')

# print '{} uids read'.format(len(uids))

time: 640 µs


In [40]:
# uids[:10]

time: 847 µs


In [41]:
# requests_per_second = 100

# for uid in uids:
# #     client.calculate_async(uid, months=['01'])
#     client.calculate_async(uid)
#     time.sleep(1.0/requests_per_second)  

time: 837 µs
